In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy import stats
import folium, fiona, branca

In [2]:
gdf1 = gpd.read_file("bnd/bnd/boundryl.shp")

In [3]:
gdf1.head()

,F_CODE,FEATURE_AT,ACC,ACCURACY_C,BST,BOUNDARY_S,NM3,NAME_3,NM4,NAME_4,USE,USAGE,geometry
0,FA000,Administrative Boundary,1,Accurate,1,Definite,99999,ECUADOR,99999,PERU-COLOMBIA,23,International,"LINESTRING (594039.080 9721055.330, 594079.730..."


In [2]:
# Crear un geodataframe que tenga como geometry los poligonos de las parroquias de la ciudad de Guayaquil. 
# Además, debe tenen una columna Name con el nombre de la parroquia, y debe tener una columna preciomt2 con el
# precio por metro cuadrado promedio en el que se venden las casas en cada parroquia.
# Asegúrese de eliminar outliers previo a realizar este análisis
# Su geodataframe guardelo en la variable gdf y ejecute las siguiente celdas:

In [4]:
df = pd.read_csv("../output/data_casas.csv")

In [5]:
df.head()

,antigüedad,cuartos,cuartosdebaño,estacionamiento,metroscuadradostotales,precio,titulo,latitud,longitud,preciomt2
0,5,4,4,0,480,565000,Arrecife - Punta Mocoli Design By Jannina Cabal,-2.115345,-79.856163,1177.083333
1,0,4,3,2,400,370000,Vendo Casa Moderna de Estreno en Isla Mocoli,-2.102443,-79.867073,925.000000
2,20,4,4,3,244,140000,Venta de Casa Cdla. del Biess Remodelada - Sec...,-2.230328,-79.902163,573.770492
3,0,0,4,2,1,259000,Se Vende Proyecto en Isla Celeste Reserva Solo...,-2.071614,-79.842065,259000.000000
4,0,4,4,0,245,245000,Isla Celeste 243 m² Const y 225 m² Terreno Her...,-2.079416,-79.837993,1000.000000


In [16]:
df = df.loc[df["metroscuadradostotales"]>0]

In [17]:
df["preciomt2"].mean()

1510.19404344291

In [20]:
df = df.loc[np.abs(stats.zscore(df["preciomt2"]))<3]

In [22]:
df.reset_index(drop=True,inplace=True)

In [23]:
df.head()

,antigüedad,cuartos,cuartosdebaño,estacionamiento,metroscuadradostotales,precio,titulo,latitud,longitud,preciomt2
0,5,4,4,0,480,565000,Arrecife - Punta Mocoli Design By Jannina Cabal,-2.115345,-79.856163,1177.083333
1,0,4,3,2,400,370000,Vendo Casa Moderna de Estreno en Isla Mocoli,-2.102443,-79.867073,925.000000
2,20,4,4,3,244,140000,Venta de Casa Cdla. del Biess Remodelada - Sec...,-2.230328,-79.902163,573.770492
3,0,4,4,0,245,245000,Isla Celeste 243 m² Const y 225 m² Terreno Her...,-2.079416,-79.837993,1000.000000
4,0,0,4,2,1090,650000,"Casa 1 Planta Al Lago - 4 D - $650,000",-2.142226,-79.864596,596.330275


In [25]:
gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df["longitud"],df["latitud"]))

In [27]:
gdf.head()

,antigüedad,cuartos,cuartosdebaño,estacionamiento,metroscuadradostotales,precio,titulo,latitud,longitud,preciomt2,geometry
0,5,4,4,0,480,565000,Arrecife - Punta Mocoli Design By Jannina Cabal,-2.115345,-79.856163,1177.083333,POINT (-79.85616 -2.11535)
1,0,4,3,2,400,370000,Vendo Casa Moderna de Estreno en Isla Mocoli,-2.102443,-79.867073,925.000000,POINT (-79.86707 -2.10244)
2,20,4,4,3,244,140000,Venta de Casa Cdla. del Biess Remodelada - Sec...,-2.230328,-79.902163,573.770492,POINT (-79.90216 -2.23033)
3,0,4,4,0,245,245000,Isla Celeste 243 m² Const y 225 m² Terreno Her...,-2.079416,-79.837993,1000.000000,POINT (-79.83799 -2.07942)
4,0,0,4,2,1090,650000,"Casa 1 Planta Al Lago - 4 D - $650,000",-2.142226,-79.864596,596.330275,POINT (-79.86460 -2.14223)


In [26]:
gdf.to_file("../output/casas.geojson", driver = "GeoJSON")

In [28]:
parrgdf = gpd.read_file("parroquias.geojson")

In [29]:
parrgdf.head()

,stroke,stroke-width,stroke-opacity,fill,fill-opacity,Name,Description,geometry
0,#555555,2.0,1.0,#555555,0.5,Parroquia Pedro Carbo,Parroquia Carbo,"POLYGON ((-79.88080 -2.18323, -79.88219 -2.183..."
1,None,NaN,NaN,None,NaN,Parroquia Roca,Parroquia Roca,"POLYGON ((-79.88843 -2.18336, -79.88825 -2.183..."
2,None,NaN,NaN,None,NaN,Parroquia Sucre,Parroquia Sucre,"POLYGON ((-79.89893 -2.19582, -79.90201 -2.205..."
3,None,NaN,NaN,None,NaN,Parroquia Urdaneta,Parroquia Urdaneta,"POLYGON ((-79.91227 -2.19029, -79.91238 -2.190..."
4,None,NaN,NaN,None,NaN,Parroquia Garcia Moreno,Parroquia García Moreno,"POLYGON ((-79.90616 -2.21938, -79.90610 -2.219..."


In [30]:
parrgdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [31]:
gdf.crs

In [32]:
gdf.crs = parrgdf.crs

In [33]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [42]:
gdf.shape

(4148, 11)

In [34]:
provgdf = gpd.read_file("provincial.json")

In [35]:
provgdf.head()

,COUNT,NOM_PROVIN,COD_PROVIN,AREA,PERIMETER,CAPITAL_PR,REGION,geometry
0,14,AZUAY,01,7.747665e+09,615749.045,CUENCA,SIERRA,"POLYGON ((681167.760 9717568.917, 681154.046 9..."
1,7,BOLIVAR,02,3.956364e+09,442149.525,GUARANDA,SIERRA,"POLYGON ((708145.497 9756730.520, 708337.417 9..."
2,7,CAÑAR,03,3.166640e+09,368897.777,AZOGUEZ,SIERRA,"POLYGON ((770341.141 9717372.806, 767272.809 9..."
3,6,CARCHI,04,3.758125e+09,424954.456,TULCAN,SIERRA,"POLYGON ((854940.686 10039749.734, 855464.962 ..."
4,7,COTOPAXI,05,5.959799e+09,417072.442,LATACUNGA,SIERRA,"POLYGON ((790092.830 9891700.437, 790138.144 9..."


In [37]:
provgdf.crs = gdf.crs

In [38]:
provgdf.head()

,COUNT,NOM_PROVIN,COD_PROVIN,AREA,PERIMETER,CAPITAL_PR,REGION,geometry
0,14,AZUAY,01,7.747665e+09,615749.045,CUENCA,SIERRA,"POLYGON ((681167.760 9717568.917, 681154.046 9..."
1,7,BOLIVAR,02,3.956364e+09,442149.525,GUARANDA,SIERRA,"POLYGON ((708145.497 9756730.520, 708337.417 9..."
2,7,CAÑAR,03,3.166640e+09,368897.777,AZOGUEZ,SIERRA,"POLYGON ((770341.141 9717372.806, 767272.809 9..."
3,6,CARCHI,04,3.758125e+09,424954.456,TULCAN,SIERRA,"POLYGON ((854940.686 10039749.734, 855464.962 ..."
4,7,COTOPAXI,05,5.959799e+09,417072.442,LATACUNGA,SIERRA,"POLYGON ((790092.830 9891700.437, 790138.144 9..."


In [40]:
dfgeo = gpd.sjoin(gdf,parrgdf,how = "inner", op = "within")

c:\users\eduardo\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2246: RuntimeWarning: invalid value encountered in find_contains (vectorized)
  outputs = ufunc(*inputs)


In [41]:
dfgeo.head()

,antigüedad,cuartos,cuartosdebaño,estacionamiento,metroscuadradostotales,precio,titulo,latitud,longitud,preciomt2,geometry,index_right,stroke,stroke-width,stroke-opacity,fill,fill-opacity,Name,Description
2,20,4,4,3,244,140000,Venta de Casa Cdla. del Biess Remodelada - Sec...,-2.230328,-79.902163,573.770492,POINT (-79.90216 -2.23033),13,None,NaN,NaN,None,NaN,Parroquia Ximena,Parroquia Ximena
35,7,9,6,0,104,135000,Casa Rentera con Cinco Departamentos en Huanca...,-2.234931,-79.900271,1298.076923,POINT (-79.90027 -2.23493),13,None,NaN,NaN,None,NaN,Parroquia Ximena,Parroquia Ximena
60,35,4,4,0,120,52000,Pradera 1 - Villa 4 Dorm. 4 Baños,-2.238747,-79.891240,433.333333,POINT (-79.89124 -2.23875),13,None,NaN,NaN,None,NaN,Parroquia Ximena,Parroquia Ximena
300,28,4,2,0,150,76000,Casa en Venta en Floresta Rentera 150 m²,-2.253352,-79.884283,506.666667,POINT (-79.88428 -2.25335),13,None,NaN,NaN,None,NaN,Parroquia Ximena,Parroquia Ximena
322,10,9,7,0,192,96000,Vendo Casa 2 Plantas con Local y 2 Departamentos,-2.266480,-79.897230,500.000000,POINT (-79.89723 -2.26648),13,None,NaN,NaN,None,NaN,Parroquia Ximena,Parroquia Ximena


In [43]:
dfgeo.shape

(1615, 19)

In [49]:
dfprec = dfgeo.loc[:,["titulo","preciomt2","Name","geometry"]].groupby("Name")["preciomt2"].mean().reset_index()

In [50]:
dfprec

,Name,preciomt2
0,Parroquia Ayacucho,459.866248
1,Parroquia Bolivar,608.024524
2,Parroquia Febres Cordero,845.369274
3,Parroquia Garcia Moreno,921.242892
4,Parroquia Letamendi,750.697339
5,Parroquia Nueve de Octubre,754.090717
6,Parroquia Olmedo,832.634102
7,Parroquia Pascuales,831.343307
8,Parroquia Pedro Carbo,796.891566
9,Parroquia Roca,767.458014


In [58]:
parrgdf = parrgdf.merge(dfprec, on = "Name")

In [59]:
parrgdf.to_file("output.geojson",driver = "GeoJSON")

In [60]:
parrgdf.head()

,stroke,stroke-width,stroke-opacity,fill,fill-opacity,Name,Description,geometry,preciomt2
0,#555555,2.0,1.0,#555555,0.5,Parroquia Pedro Carbo,Parroquia Carbo,"POLYGON ((-79.88080 -2.18323, -79.88219 -2.183...",796.891566
1,None,NaN,NaN,None,NaN,Parroquia Roca,Parroquia Roca,"POLYGON ((-79.88843 -2.18336, -79.88825 -2.183...",767.458014
2,None,NaN,NaN,None,NaN,Parroquia Sucre,Parroquia Sucre,"POLYGON ((-79.89893 -2.19582, -79.90201 -2.205...",532.432936
3,None,NaN,NaN,None,NaN,Parroquia Urdaneta,Parroquia Urdaneta,"POLYGON ((-79.91227 -2.19029, -79.91238 -2.190...",498.525361
4,None,NaN,NaN,None,NaN,Parroquia Garcia Moreno,Parroquia García Moreno,"POLYGON ((-79.90616 -2.21938, -79.90610 -2.219...",921.242892


In [56]:
#crea el mapa con un tile service
centroid=parrgdf.geometry.centroid
mapbox_api_key = r"pk.eyJ1Ijoic3RhbWVuIiwiYSI6IlpkZEtuS1EifQ.jiH_c9ShtBwtqH9RdG40mw"; tileset_ID_str = "streets-v11"; tilesize_pixels = "256";
m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=12,
             tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}",
             attr='MAPBOX')

In [61]:
field = "preciomt2"
label = "precio por metro cuadrado"
colormap = branca.colormap.linear.YlOrBr_04.to_step(4).scale(parrgdf[field].min(),parrgdf[field].max())
colormap.caption = label
layer = folium.GeoJson('output.geojson',
        name=label,
        style_function=lambda x: {"weight":2, 'color':'black','fillColor':colormap(x['properties'][field]), 'fillOpacity':1},
        highlight_function=lambda x: {'weight':5, 'color':'black'},
        smooth_factor=2.0,
        tooltip=folium.features.GeoJsonTooltip(fields=['Name',field],
                                      aliases=['Name',label], 
                                      labels=True, 
                                      sticky=True,
                                      localize=True
                                     ),
        show = True,
        overlay = True
        )
m.add_child(layer)
m.add_child(colormap)

In [62]:
m.save('choropleth.html')